In [1]:
!pip install eli5

  Created wheel for tabulate: filename=tabulate-0.8.6-cp37-none-any.whl size=23279 sha256=28475dedce4269fd3bac344fec560e9a7e357e4663e0b79755eb95adcdabaf6c
  Stored in directory: C:\Users\benia\AppData\Local\pip\Cache\wheels\9c\9b\f4\eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e
Successfully built tabulate


In [2]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyRegressor
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import eli5 
from eli5.sklearn import PermutationImportance


Using TensorFlow backend.


In [3]:
cd ..

C:\Users\benia\Desktop\Machine_Learning\Prediction price cars


In [4]:
cd data

C:\Users\benia\Desktop\Machine_Learning\Prediction price cars\data


In [5]:
df = pd.read_hdf('car.h5')

In [13]:
df.head()

,breadcrumb,created_at,price_currency,price_details,price_value,seller_address,seller_name,seller_type,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,feature_kurtyny-powietrzne,feature_klimatyzacja-dwustrefowa,feature_światła-led,feature_czujnik-zmierzchu,feature_elektrycznie-ustawiane-lusterka,feature_asr-(kontrola-trakcji),feature_poduszka-powietrzna-kierowcy,feature_cd,feature_elektryczne-szyby-przednie,feature_poduszka-powietrzna-pasażera,feature_system-start-stop,feature_światła-do-jazdy-dziennej,feature_komputer-pokładowy,feature_elektryczne-szyby-tylne,feature_klimatyzacja-manualna,feature_tapicerka-welurowa,feature_czujnik-deszczu,feature_światła-przeciwmgielne,feature_ogrzewanie-postojowe,feature_radio-niefabryczne,feature_regulowane-zawieszenie,feature_ogranicznik-prędkości,feature_zmieniarka-cd,feature_szyberdach,feature_isofix,feature_asystent-pasa-ruchu,feature_alufelgi,feature_bluetooth,feature_nawigacja-gps,feature_asystent-parkowania,feature_wspomaganie-kierownicy,feature_podgrzewana-przednia-szyba,feature_przyciemniane-szyby,feature_elektrycznie-ustawiane-fotele,feature_klimatyzacja-czterostrefowa,feature_tuner-tv,feature_poduszki-boczne-przednie,feature_tempomat-aktywny,feature_klimatyzacja-automatyczna,feature_poduszki-boczne-tylne,feature_radio-fabryczne,feature_odtwarzacz-dvd,feature_czujniki-parkowania-tylne,feature_łopatki-zmiany-biegów,feature_centralny-zamek,feature_dach-panoramiczny,feature_kamera-cofania,feature_immobilizer,feature_czujnik-martwego-pola,feature_gniazdo-sd,...,param_metallic,param_leasing-concession,param_color,param_financing-option,param_original-owner,param_vat-marża,param_kategoria,param_co2-emissions,param_leasing,param_mileage,param_zarejestrowany-jako-zabytek,param_napęd,param_wartość-wykupu,param_typ,param_metalik,param_pierwszy-właściciel,param_skrzynia-biegów,param_vat-discount,param_no-accident,param_liczba-pozostałych-rat,param_particle-filter,param_zarejestrowany-w-polsce,param_oferta-od,param_moc,param_make,param_kraj-pochodzenia,param_engine-power,param_year,param_serwisowany-w-aso,param_rodzaj-paliwa,param_możliwość-finansowania,param_registered-in-poland,param_transmission,param_miesięczna-rata,param_kod-silnika,param_version,param_opłata-początkowa,param_model-pojazdu,param_pojemność-skokowa,param_kierownica-po-prawej-(anglik),param_fuel-type,param_vat-free,param_acrylic,param_damaged,param_door-count,param_tuning,param_filtr-cząstek-stałych,param_nr-of-seats,param_kolor,param_engine-capacity,param_pearl,param_stan,param_wersja,param_emisja-co2,param_body-type,param_matowy,param_bezwypadkowy,param_akryl-(niemetalizowany),param_monthly-payment-value,car_id
0,"[Osobowe, Dacia, Logan, Dacia Logan II (2012-)]","13:23, 5 marca 2018",PLN,Cena Brutto,45100.0,"Krasne 7A - 36-007 Krasne, rzeszowski, Podkarp...",AUTO SPEKTRUM,Dealer,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,...,None,None,None,None,None,None,Osobowe,None,None,None,None,None,None,Sedan,Tak,None,Manualna,None,None,None,None,None,Firmy,90 KM,None,None,None,None,None,Benzyna+LPG,None,None,None,None,None,None,None,Logan,898 cm3,None,None,None,None,None,None,None,None,None,Inny kolor,None,None,Nowe,II (2012-),None,None,None,Tak,None,None,0
2,"[Osobowe, Peugeot, 3008, Peugeot 3008 I (2009-...","15:48, 5 marca 2018",PLN,"Cena Brutto, Do negocjacji, Faktura VAT",28000.0,"Tomaszów Mazowiecki, tomaszowski, Łódzkie",None,Osoba prywatna,False,False,False,True,False,False,True,True,True,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,True,False,False,...,

In [14]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106494 entries, 0 to 160615
Columns: 155 entries, breadcrumb to car_id
dtypes: bool(70), float64(1), int64(1), object(83)
memory usage: 77.0+ MB


In [16]:
df.select_dtypes(np.number).columnsumns

Index(['price_value', 'car_id'], dtype='object')

In [22]:
X = df[["car_id"]]
y= df["price_value"]
model = DummyRegressor()
model.fit(X,y)
y_pred = model.predict(X)
mae(y,y_pred)

39465.934630440985

In [24]:
y_pred

array([47534.13743704, 47534.13743704, 47534.13743704, ...,
       47534.13743704, 47534.13743704, 47534.13743704])

In [25]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [26]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [6]:
df = df[df['price_currency'] != "EUR"]
df.shape


(106290, 155)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106290 entries, 0 to 160615
Columns: 155 entries, breadcrumb to car_id
dtypes: bool(70), float64(1), int64(1), object(83)
memory usage: 76.8+ MB


In [32]:
for feat in df.columns:
    print(feat)

breadcrumb
created_at
price_currency
price_details
price_value
seller_address
seller_name
seller_type
feature_czujniki-parkowania-przednie
feature_poduszka-powietrzna-chroniąca-kolana
feature_kurtyny-powietrzne
feature_klimatyzacja-dwustrefowa
feature_światła-led
feature_czujnik-zmierzchu
feature_elektrycznie-ustawiane-lusterka
feature_asr-(kontrola-trakcji)
feature_poduszka-powietrzna-kierowcy
feature_cd
feature_elektryczne-szyby-przednie
feature_poduszka-powietrzna-pasażera
feature_system-start-stop
feature_światła-do-jazdy-dziennej
feature_komputer-pokładowy
feature_elektryczne-szyby-tylne
feature_klimatyzacja-manualna
feature_tapicerka-welurowa
feature_czujnik-deszczu
feature_światła-przeciwmgielne
feature_ogrzewanie-postojowe
feature_radio-niefabryczne
feature_regulowane-zawieszenie
feature_ogranicznik-prędkości
feature_zmieniarka-cd
feature_szyberdach
feature_isofix
feature_asystent-pasa-ruchu
feature_alufelgi
feature_bluetooth
feature_nawigacja-gps
feature_asystent-parkowania
fe

In [33]:
df.corr()

,price_value,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,feature_kurtyny-powietrzne,feature_klimatyzacja-dwustrefowa,feature_światła-led,feature_czujnik-zmierzchu,feature_elektrycznie-ustawiane-lusterka,feature_asr-(kontrola-trakcji),feature_poduszka-powietrzna-kierowcy,feature_cd,feature_elektryczne-szyby-przednie,feature_poduszka-powietrzna-pasażera,feature_system-start-stop,feature_światła-do-jazdy-dziennej,feature_komputer-pokładowy,feature_elektryczne-szyby-tylne,feature_klimatyzacja-manualna,feature_tapicerka-welurowa,feature_czujnik-deszczu,feature_światła-przeciwmgielne,feature_ogrzewanie-postojowe,feature_radio-niefabryczne,feature_regulowane-zawieszenie,feature_ogranicznik-prędkości,feature_zmieniarka-cd,feature_szyberdach,feature_isofix,feature_asystent-pasa-ruchu,feature_alufelgi,feature_bluetooth,feature_nawigacja-gps,feature_asystent-parkowania,feature_wspomaganie-kierownicy,feature_podgrzewana-przednia-szyba,feature_przyciemniane-szyby,feature_elektrycznie-ustawiane-fotele,feature_klimatyzacja-czterostrefowa,feature_tuner-tv,feature_poduszki-boczne-przednie,feature_tempomat-aktywny,feature_klimatyzacja-automatyczna,feature_poduszki-boczne-tylne,feature_radio-fabryczne,feature_odtwarzacz-dvd,feature_czujniki-parkowania-tylne,feature_łopatki-zmiany-biegów,feature_centralny-zamek,feature_dach-panoramiczny,feature_kamera-cofania,feature_immobilizer,feature_czujnik-martwego-pola,feature_gniazdo-sd,feature_podgrzewane-lusterka-boczne,feature_podgrzewane-przednie-siedzenia,feature_hak,feature_abs,feature_wielofunkcyjna-kierownica,feature_światła-xenonowe,feature_gniazdo-usb,feature_tapicerka-skórzana,feature_gniazdo-aux,feature_esp-(stabilizacja-toru-jazdy),feature_alarm,feature_podgrzewane-tylne-siedzenia,feature_mp3,feature_tempomat,feature_hud-(wyświetlacz-przezierny),feature_elektrochromatyczne-lusterko-wsteczne,feature_relingi-dachowe,feature_elektrochromatyczne-lusterka-boczne,car_id
price_value,1.000000,0.401060,0.205070,0.135849,0.139081,0.435773,0.283763,0.012686,0.163640,-0.108947,-0.093696,-0.056233,-0.069872,0.423519,0.335418,0.097895,0.171859,-0.190123,-0.178328,0.270575,0.028003,0.083008,-0.060114,0.299238,0.229570,0.026977,0.122803,0.178009,0.379528,0.170659,0.367045,0.362353,0.307270,-0.113039,0.072339,0.203206,0.295236,0.234764,0.103067,0.067334,0.306447,0.157898,0.081524,0.034727,0.183535,0.293714,0.426733,-0.080059,0.208939,0.465398,-0.045089,0.347950,0.257579,0.127983,0.275678,-0.079379,-0.072729,0.201730,0.149560,0.338116,0.273029,0.213973,0.162696,0.140146,0.248966,0.181333,0.154060,0.252824,0.222239,0.059940,0.182745,-0.001318
feature_czujniki-parkowania-przednie,0.401060,1.000000,0.305894,0.310377,0.360090,0.455532,0.494506,0.211574,0.312993,0.133931,0.118244,0.171818,0.162316,0.443050,0.419765,0.284882,0.409617,-0.262953,-0.103948,0.494395,0.233958,0.138716,-0.083251,0.272880,0.304898,0.201092,0.194310,0.295570,0.340878,0.359127,0.474570,0.537735,0.410409,0.138761,0.200758,0.324595,0.440707,0.226779,0.122345,0.231663,0.311433,0.350736,0.253915,0.222293,0.306324,0.630617,0.363718,0.154035,0.234058,0.397322,0.163540,0.337234,0.356971,0.306951,0.441566,0.025823,0.160157,0.369269,0.402609,0.363854,0.378708,0.336709,0.316082,0.270774,0.229271,0.281314,0.361798,0.196624,0.423919,0.202572,0.341219,0.004178
feature_poduszka-powietrzna-chroniąca-kolana,0.205070,0.305894,1.000000,0.328987,0.232617,0.311095,0.332930,0.178159,0.269537,0.144232,0.137925,0.148160,0.162941,0.271367,0.304531,0.238710,0.275071,-0.149111,0.020182,0.324748,0.198529,0.105331,-0.064845,0.170584,0.219186,0.152462,0.116908,0.267779,0.243543,0.233506,0.333499,0.277070,0.250245,0.134579,0.229701,0.241531,0.264885,0.158666,0.102677,0.326110,0.237273,0.240568,0.366624,0.186944,0.224116,0.263817,0.247200,0.129676,0.151958,0.274247,0.148900,0.234990,0.319812,0.266782,0.267047,0.039905,0.140657,0.284169,0.197242,0.318875,0.216625,0.288442,0.263019,0.213204,0.161289,0.261808,0.239935,0.122144,0.329993,0.

In [7]:
SUFFIX_CAT = "_cat"
for feat in df.columns:
    if isinstance(df[feat][0],list): continue
    
    factorized_values = df[feat].factorize()[0]
    
    if SUFFIX_CAT in feat:
        df[feat] = factorized_values
    else:
        df[feat + SUFFIX_CAT] = factorized_values


In [19]:
cat_feats

['created_at_cat',
 'seller_address_cat',
 'seller_name_cat',
 'seller_type_cat',
 'feature_czujniki-parkowania-przednie_cat',
 'feature_poduszka-powietrzna-chroniąca-kolana_cat',
 'feature_kurtyny-powietrzne_cat',
 'feature_klimatyzacja-dwustrefowa_cat',
 'feature_światła-led_cat',
 'feature_czujnik-zmierzchu_cat',
 'feature_elektrycznie-ustawiane-lusterka_cat',
 'feature_asr-(kontrola-trakcji)_cat',
 'feature_poduszka-powietrzna-kierowcy_cat',
 'feature_cd_cat',
 'feature_elektryczne-szyby-przednie_cat',
 'feature_poduszka-powietrzna-pasażera_cat',
 'feature_system-start-stop_cat',
 'feature_światła-do-jazdy-dziennej_cat',
 'feature_komputer-pokładowy_cat',
 'feature_elektryczne-szyby-tylne_cat',
 'feature_klimatyzacja-manualna_cat',
 'feature_tapicerka-welurowa_cat',
 'feature_czujnik-deszczu_cat',
 'feature_światła-przeciwmgielne_cat',
 'feature_ogrzewanie-postojowe_cat',
 'feature_radio-niefabryczne_cat',
 'feature_regulowane-zawieszenie_cat',
 'feature_ogranicznik-prędkości_cat',

In [8]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [29]:
X = df[['param_faktura-vat_cat','param_faktura-vat_cat','param_stan_cat','param_rok-produkcji_cat','param_moc_cat','feature_kamera-cofania_cat']].values
y = df["price_value"].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model,X,y,cv=3,scoring="neg_mean_absolute_error")
np.mean(scores)

-23838.648478259605

In [10]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X,y)
imp = PermutationImportance(m,random_state=0).fit(X,y)
eli5.show_weights(imp,feature_names =cat_feats)

Weight,Feature
0.2533 ± 0.0040,param_napęd_cat
0.2008 ± 0.0047,param_faktura-vat_cat
0.1943 ± 0.0088,param_stan_cat
0.1423 ± 0.0086,param_rok-produkcji_cat
0.0629 ± 0.0047,param_moc_cat
0.0424 ± 0.0014,feature_kamera-cofania_cat
0.0412 ± 0.0008,param_skrzynia-biegów_cat
0.0286 ± 0.0037,param_marka-pojazdu_cat
0.0191 ± 0.0022,param_pojemność-skokowa_cat
0.0163 ± 0.0005,feature_bluetooth_cat


In [13]:
df.columns.values

array(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       'feature_kurtyny-powietrzne', 'feature_klimatyzacja-dwustrefowa',
       'feature_światła-led', 'feature_czujnik-zmierzchu',
       'feature_elektrycznie-ustawiane-lusterka',
       'feature_asr-(kontrola-trakcji)',
       'feature_poduszka-powietrzna-kierowcy', 'feature_cd',
       'feature_elektryczne-szyby-przednie',
       'feature_poduszka-powietrzna-pasażera',
       'feature_system-start-stop', 'feature_światła-do-jazdy-dziennej',
       'feature_komputer-pokładowy', 'feature_elektryczne-szyby-tylne',
       'feature_klimatyzacja-manualna', 'feature_tapicerka-welurowa',
       'feature_czujnik-deszczu', 'feature_światła-przeciwmgielne',
       'feature_ogrzewanie-postojowe', 'feature_radio-niefabryczne',
       'feature_regulowane-zaw